In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

0 completed
1 completed
2 completed
3 completed
4 completed
5 completed
6 completed


In [5]:
print(train)

                                      image label
0      images/train\0\Training_10002154.jpg     0
1      images/train\0\Training_10031781.jpg     0
2      images/train\0\Training_10055498.jpg     0
3      images/train\0\Training_10059941.jpg     0
4      images/train\0\Training_10078021.jpg     0
...                                     ...   ...
28704  images/train\6\Training_99946015.jpg     6
28705  images/train\6\Training_99949469.jpg     6
28706  images/train\6\Training_99957517.jpg     6
28707  images/train\6\Training_99966135.jpg     6
28708  images/train\6\Training_99982465.jpg     6

[28709 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

0 completed
1 completed
2 completed
3 completed
4 completed
5 completed
6 completed


In [7]:
print(test)
print(test['image'])

                                       image label
0     images/test\0\PrivateTest_10086748.jpg     0
1     images/test\0\PrivateTest_10767287.jpg     0
2     images/test\0\PrivateTest_11123843.jpg     0
3     images/test\0\PrivateTest_11164800.jpg     0
4     images/test\0\PrivateTest_11239107.jpg     0
...                                      ...   ...
7173   images/test\6\PublicTest_99365116.jpg     6
7174   images/test\6\PublicTest_99509833.jpg     6
7175   images/test\6\PublicTest_99607072.jpg     6
7176   images/test\6\PublicTest_99646813.jpg     6
7177   images/test\6\PublicTest_99743122.jpg     6

[7178 rows x 2 columns]
0       images/test\0\PrivateTest_10086748.jpg
1       images/test\0\PrivateTest_10767287.jpg
2       images/test\0\PrivateTest_11123843.jpg
3       images/test\0\PrivateTest_11164800.jpg
4       images/test\0\PrivateTest_11239107.jpg
                         ...                  
7173     images/test\6\PublicTest_99365116.jpg
7174     images/test\6\PublicTest_

In [8]:
from tqdm.notebook import tqdm

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [10]:
# for image_path in tqdm(images):
#     print(f"Processing image: {image_path}")
#     img = load_img(image_path, grayscale=True)

In [11]:
train_features = extract_features(train['image']) 

  0%|          | 0/28709 [00:00<?, ?it/s]

C:\Users\mamta\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [12]:
test_features = extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [13]:
x_train = train_features/255.0
x_test = test_features/255.0

In [14]:
# print("Unique labels in y_train:", np(y_train))

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
# le = LabelEncoder()
# le.fit(train['label'])

In [17]:
# Check data type of labels in train set
print("Data type of labels in train set:", train['label'].dtype)

# Print some sample values from train set
print("Sample values from train set:", train['label'].unique())


Data type of labels in train set: object
Sample values from train set: ['0' '1' '2' '3' '4' '5' '6']


In [18]:
y_train = train['label'].values
y_test = test['label'].values

In [19]:
print("Unique labels in y_train:", np.unique(y_train))

Unique labels in y_train: ['0' '1' '2' '3' '4' '5' '6']


In [20]:
# y_train = to_categorical(y_train,num_classes = 7)
# y_test = to_categorical(y_test,num_classes = 7)

In [21]:
print("Unique labels in y_train:", np.unique(y_train))


Unique labels in y_train: ['0' '1' '2' '3' '4' '5' '6']


In [22]:
# from collections import Counter

# # Convert each NumPy array in y_train to a tuple
# y_train_tuples = [tuple(arr) for arr in y_train]

# # Use Counter on the tuples
# label_counts = Counter(y_train_tuples)
# print(label_counts)


In [23]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [24]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy' )

In [25]:
# Example for features
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Example for labels (if using integer labels)
y_train = y_train.astype('int')
y_test = y_test.astype('int')


In [26]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100


225/225 [==============================] - 150s 656ms/step - loss: 1.8213 - accuracy: 0.2470 - val_loss: 1.8147 - val_accuracy: 0.2471
Epoch 2/100
225/225 [==============================] - 147s 654ms/step - loss: 1.7869 - accuracy: 0.2565 - val_loss: 1.7315 - val_accuracy: 0.2947
Epoch 3/100
225/225 [==============================] - 282s 1s/step - loss: 1.6971 - accuracy: 0.3076 - val_loss: 1.5943 - val_accuracy: 0.3749
Epoch 4/100
225/225 [==============================] - 156s 692ms/step - loss: 1.5750 - accuracy: 0.3796 - val_loss: 1.4652 - val_accuracy: 0.4277
Epoch 5/100
225/225 [==============================] - 157s 696ms/step - loss: 1.4755 - accuracy: 0.4267 - val_loss: 1.3569 - val_accuracy: 0.4760
Epoch 6/100
225/225 [==============================] - 166s 738ms/step - loss: 1.4232 - accuracy: 0.4509 - val_loss: 1.3093 - val_accuracy: 0.4964
Epoch 7/100
225/225 [==============================] - 168s 746ms/step - loss: 1.3801 - accuracy: 0.4699 - val_loss: 1.

In [27]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

C:\Users\mamta\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
from keras.models import model_from_json

In [29]:
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")

FileNotFoundError: [Errno 2] No such file or directory: 'facialemotionmodel.json'

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [ ]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/fear/2.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/disgust/299.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/happy/7.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/surprise/15.jpg'
print("original image is of surprise")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
print(model)

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

In [ ]:
model.summary()

In [ ]:
# model.save("emotiondetector.h5")